In [ ]:
import requests
import json
import pandas as pd
from pandas import json_normalize
import config
from db_manager import DBManager
import os
from rest_api import Api
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
from urllib.parse import quote_plus
from sqlalchemy import create_engine
import itertools
import logging
import pickle
import hashlib

In [ ]:
logging.basicConfig(
    format='%(levelname)s-%(lineno)s-%(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    level='INFO'
)
logger=logging.getLogger(__name__)

#### Show all columns without truncating

Today's date

In [ ]:
today = date.today()
logger.info(f'Current date: {today}')

INFO-__main__:2-Current date: 2023-11-05


Adding months to today's date. Near term travel dates 30 - 60 days ahead

In [ ]:
departure_date_1_month = today + relativedelta(months=1)
departure_date_2_month = today + relativedelta(months=2)
return_date_1_month = departure_date_2_month
return_date_2_month = today + relativedelta(months=3)
logger.info(f'Departure_dates:{departure_date_1_month} and {departure_date_2_month} \nReturn dates:{return_date_1_month} and {return_date_2_month}')

INFO-__main__:5-Departure_dates:2023-12-05 and 2024-01-05 
Return dates:2024-01-05 and 2024-02-05


Get the nearby festival departure dates by start_date from the public.indian_holidays table

In [ ]:
# credentials=config.pg_credentials
# db = DBManager(credentials=credentials)
# df_departure_arrival_dates = db.run_query(query_file_name='festival_dates')

In [ ]:
#df_departure_arrival_dates 

Apply the combination of params to get data for all possible combinations of depart, arrival dates and depart arrival airports

Combination itertools usage. Im calculating the total combinations by multiplying the number of possibilities (unique values) for each variable<br />
Departure airport: 2 possibilities<br />
Arrival airport: 4 possibilities<br />
Departure date: 2 possibilities<br />
Arrival date: 1 possibilities<br />
Total combinations = 2 x 4 x 2 x 1 = 16 combinations

#### Travel payouts API to get flight search results for each combination in the list 

--------------------------------------------------------

API test for amadeus

POST request to the Amadeus authorization server to get the access token

In [ ]:
response = requests.post(url=config.url_token, headers=config.headers_token, data=config.data)

if response.status_code == 200:
  # API call succeeded
  token = response.json()['access_token'] 
  logger.info('Successful access token generation')
else:
  # API call failed
  print(response.text)

INFO-__main__:6-Successful access token generation


In [ ]:
#Combinations of destination and origin airport codes
origins = config.params['originLocationCode']
destinations = config.params['destinationLocationCode']
departure_dates=[departure_date_1_month,departure_date_2_month]
# return_dates=[return_date_1_month,return_date_2_month]

Creating combinations of Origin and Destination to be passed as parameters to the api call

trial

In [ ]:
headers = {
  'client_id': config.client_id, 
  'client_secret': config.client_secret,
  'Authorization': f'Bearer {token}'
  }

url = config.url

response = []
counter = 0

for origin, dest, depart in itertools.product(origins, destinations, departure_dates):

    params={
        'originLocationCode': origin,
        'destinationLocationCode':dest, 
        'departureDate': depart, 
        'returnDate' : None,
        'adults':1,
        'children':None,
        'infants':None,
        'travelClass':None,
        'currencyCode':'EUR',
        'maxPrice' : None
        }


    #to be worked on later for by calling rest_api module
    # api = Api()
    # resp = api.make_flight_api_request(url,headers,params)
    resp = requests.get(url, headers=headers, params=params)
    counter = counter+1
    resp = resp.json()
    response.append(resp)
    logger.info(f'API call {counter} succeeded for flight offers')
    # else:
    #     # API call failed
    #     logger.info('API call failed')


INFO-__main__:35-API call 1 succeeded for flight offers
INFO-__main__:35-API call 2 succeeded for flight offers
INFO-__main__:35-API call 3 succeeded for flight offers
INFO-__main__:35-API call 4 succeeded for flight offers
INFO-__main__:35-API call 5 succeeded for flight offers
INFO-__main__:35-API call 6 succeeded for flight offers
INFO-__main__:35-API call 7 succeeded for flight offers
INFO-__main__:35-API call 8 succeeded for flight offers
INFO-__main__:35-API call 9 succeeded for flight offers
INFO-__main__:35-API call 10 succeeded for flight offers
INFO-__main__:35-API call 11 succeeded for flight offers
INFO-__main__:35-API call 12 succeeded for flight offers
INFO-__main__:35-API call 13 succeeded for flight offers
INFO-__main__:35-API call 14 succeeded for flight offers
INFO-__main__:35-API call 15 succeeded for flight offers
INFO-__main__:35-API call 16 succeeded for flight offers
INFO-__main__:35-API call 17 succeeded for flight offers
INFO-__main__:35-API call 18 succeeded f

### Saving the Global variable to a file

### Creating the payload for real time price script

In [ ]:

# payload = {
#   "data": {
#     "type": "flight-offers-pricing",
#     "flightOffers": [], 
#     "travelers": []
#   }
# }

# for r in response:

#   # Extract flight offer data
#   offers = r['data']
  
#   # Add offers to payload
#   payload['data']['flightOffers'].extend(offers) 

# # Convert payload to JSON string for request 
# json_payload = json.dumps(payload)

In [ ]:
# flight_search_response =json_payload

In [ ]:
# Save the variable to a file
# with open('global_variable.pkl', 'wb') as file:
#     pickle.dump(flight_search_response, file)

Convert that sample nested JSON data into separate DataFrames

Repeat the same above procedure for itineraries and traveler_pricing

### Offers

In [ ]:
offer_df = pd.DataFrame()

for i in range(24):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], record_path=['itineraries'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    offer_df = pd.concat([offer_df, data_to_append], ignore_index=True)

Checking what the offer df contains

To get the first dept airport and last arrival airport looping through each row in itineraries df

In [ ]:
offer_df['first_dept_airport'] = offer_df['segments'].apply(lambda x: x[0]['departure']['iataCode']) 
offer_df['last_arr_airport'] = offer_df['segments'].apply(lambda x: x[-1]['arrival']['iataCode'])
offer_df['first_dept_time'] = offer_df['segments'].apply(lambda x: x[0]['departure']['at']) 
offer_df['last_arr_time'] = offer_df['segments'].apply(lambda x: x[-1]['arrival']['at'])

Creating a composite key

Think About a unique key that you will use to join pricing table and segments table together to get the price of the flight

In [ ]:
offer_df = offer_df.rename(columns={'id': 'offer_id'})

Trying another unique id

In [ ]:
def generate_unique_id(row):
    unique_string = f"{row['offer_id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

offer_df['unique_id'] = offer_df.apply(generate_unique_id, axis=1)

In [ ]:
offer_df=offer_df.drop(columns=['segments'],axis=1)

In [ ]:
# Define the desired column order
desired_order = ['unique_id', 'offer_id', 'source','first_dept_airport','last_arr_airport','first_dept_time','last_arr_time','duration']

# Create a new DataFrame with columns in the desired order
offer_df = offer_df[desired_order]

In [ ]:
offer_df.tail()

,unique_id,offer_id,source,first_dept_airport,last_arr_airport,first_dept_time,last_arr_time,duration
2075,46ea482110caf821a34376ab382ced51,78,GDS,MUC,MAA,2024-01-05T11:00:00,2024-01-05T23:00:00,PT7H30M
2076,fad9604dace84a83a9a6698ff4aebb0a,79,GDS,MUC,MAA,2024-01-05T11:10:00,2024-01-05T23:00:00,PT7H20M
2077,f214dc50b41a134209eca106e3bdd1d4,80,GDS,MUC,MAA,2024-01-05T09:10:00,2024-01-05T23:00:00,PT9H20M
2078,e4003355b1be87bcff88406733df3e5d,81,GDS,MUC,MAA,2024-01-05T08:05:00,2024-01-05T23:00:00,PT10H25M
2079,53b6e6d97b4973895870b531eaa045dc,82,GDS,MUC,MAA,2024-01-05T21:45:00,2024-01-06T13:30:00,PT11H15M


### Itineraries segments

In [ ]:
itineraries_df = pd.DataFrame()

for i in range(24):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], record_path=['itineraries'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    itineraries_df = pd.concat([itineraries_df, data_to_append], ignore_index=True)

In [ ]:
itineraries_df['first_dept_airport'] = itineraries_df['segments'].apply(lambda x: x[0]['departure']['iataCode']) 
itineraries_df['last_arr_airport'] = itineraries_df['segments'].apply(lambda x: x[-1]['arrival']['iataCode'])
itineraries_df['first_dept_time'] = itineraries_df['segments'].apply(lambda x: x[0]['departure']['at']) 
itineraries_df['last_arr_time'] = itineraries_df['segments'].apply(lambda x: x[-1]['arrival']['at'])

Trying to genearte unique id before exploding

In [ ]:
def generate_unique_id(row):
    unique_string = f"{row['id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

itineraries_df['unique_id_fk'] = itineraries_df.apply(generate_unique_id, axis=1)

Treating the itineraries df

Just taking the exploded df and carefully scanning each row in the top 5 rows to see what kind of values does the exploded version of offer df contains
exploded df
df2
df

In [ ]:
# Explode the 'segment' column to create separate rows for each dictionary
exploded_df = itineraries_df.explode('segments')

# Split the dictionary into columns  
df2 = exploded_df['segments'].apply(pd.Series)

# Merge the new columns back to the original DataFrame
df = exploded_df.merge(df2, right_index=True, left_index=True, suffixes=('_original', '_segments'))

# Drop the original dictionary column
df = df.drop(columns=['segments']) 
df = df.rename(columns={'duration_original': 'duration_total','id_original':'offer_id'})

In [ ]:
# Extract departure keys using apply
df['dept_airport'] = df['departure'].apply(lambda x: x['iataCode'])
df['dept_at'] = df['departure'].apply(lambda x: x['at'])
df['arrival_airport'] = df['arrival'].apply(lambda x: x['iataCode'])
df['arrival_at'] = df['arrival'].apply(lambda x: x['at'])
df['aircraft'] = df['aircraft'].apply(lambda x: x['code'])
df.head()
# Drop stops column if it exists
column_to_drop = 'stops'
if column_to_drop in df.columns:
    df.drop(column_to_drop,axis=1,inplace=True)
itineraries_df = df.drop(columns=['departure','arrival','operating','aircraft','blacklistedInEU','first_dept_airport','last_arr_airport','first_dept_time','last_arr_time'])

itineraries_df = itineraries_df.drop_duplicates()
# Reset index
itineraries_df = itineraries_df.reset_index(drop=True)
itineraries_df.head()

,duration_total,offer_id,source,unique_id_fk,carrierCode,number,duration_segments,id_segments,numberOfStops,dept_airport,dept_at,arrival_airport,arrival_at
0,PT10H35M,1,GDS,7cbbb21d4e8e94e90668e6ab656eaaf2,KL,1824,PT1H25M,100,0,BER,2023-12-05T11:50:00,AMS,2023-12-05T13:15:00
1,PT10H35M,1,GDS,7cbbb21d4e8e94e90668e6ab656eaaf2,KL,109,PT8H15M,101,0,AMS,2023-12-05T14:10:00,DEL,2023-12-06T01:55:00
2,PT16H25M,2,GDS,b66001f51ad00ed9f592f32d5e64bb81,KL,1818,PT1H25M,82,0,BER,2023-12-05T06:00:00,AMS,2023-12-05T07:25:00
3,PT16H25M,2,GDS,b66001f51ad00ed9f592f32d5e64bb81,KL,109,PT8H15M,83,0,AMS,2023-12-05T14:10:00,DEL,2023-12-06T01:55:00
4,PT25H15M,3,GDS,759e075afd7882db4d165ca69d0bd4eb,KL,1834,PT1H30M,61,0,BER,2023-12-05T21:10:00,AMS,2023-12-05T22:40:00


In [ ]:
itineraries_df['segment_key'] = itineraries_df['unique_id_fk'].astype(str) + '_' + itineraries_df['id_segments'].astype(str)

Create unique id for segments df

Renaming, reordering and changing datatypes

In [ ]:
# Rename the columns according to the SQL table schema
itineraries_df = itineraries_df.rename(columns={
    'source': 'source',
    'duration_segments': 'duration_segments',
    'duration_total': 'duration_total',
    'carrierCode': 'carrier_code',
    'number': 'number',
    'duration_segments': 'duration_segments',
    'numberOfStops': 'number_of_stops',
    'dept_airport': 'dept_airport',
    'dept_at': 'dept_at',
    'arrival_airport': 'arrival_airport',
    'arrival_at': 'arrival_at'
})

# Ensure data types are set correctly
itineraries_df['offer_id'] = itineraries_df['offer_id'].astype(int)
itineraries_df['number_of_stops'] = itineraries_df['number_of_stops'].astype(int)
itineraries_df['dept_at'] = pd.to_datetime(itineraries_df['dept_at'])
itineraries_df['arrival_at'] = pd.to_datetime(itineraries_df['arrival_at'])

now = datetime.now()
itineraries_df['incremental_day']=now
itineraries_df['incremental_day'] = pd.to_datetime(itineraries_df['incremental_day'])


In [ ]:
logger.info(f'Current date: {now}')

INFO-__main__:1-Current date: 2023-11-05 20:23:55.149435


In [ ]:
# Assuming itineraries_df is your DataFrame
new_column_order = [
    'segment_key',
    'duration_total',
    'offer_id',
    'source',
    'unique_id_fk',
    'carrier_code',
    'number',
    'duration_segments',
    'id_segments',
    'number_of_stops',
    'dept_airport',
    'dept_at',
    'arrival_airport',
    'arrival_at',
    'incremental_day'
]

# Reorder the DataFrame columns
itineraries_df = itineraries_df[new_column_order]


In [ ]:
itineraries_df.head()

,segment_key,duration_total,offer_id,source,unique_id_fk,carrier_code,number,duration_segments,id_segments,number_of_stops,dept_airport,dept_at,arrival_airport,arrival_at,incremental_day
0,7cbbb21d4e8e94e90668e6ab656eaaf2_100,PT10H35M,1,GDS,7cbbb21d4e8e94e90668e6ab656eaaf2,KL,1824,PT1H25M,100,0,BER,2023-12-05 11:50:00,AMS,2023-12-05 13:15:00,2023-11-05 20:23:55.149435
1,7cbbb21d4e8e94e90668e6ab656eaaf2_101,PT10H35M,1,GDS,7cbbb21d4e8e94e90668e6ab656eaaf2,KL,109,PT8H15M,101,0,AMS,2023-12-05 14:10:00,DEL,2023-12-06 01:55:00,2023-11-05 20:23:55.149435
2,b66001f51ad00ed9f592f32d5e64bb81_82,PT16H25M,2,GDS,b66001f51ad00ed9f592f32d5e64bb81,KL,1818,PT1H25M,82,0,BER,2023-12-05 06:00:00,AMS,2023-12-05 07:25:00,2023-11-05 20:23:55.149435
3,b66001f51ad00ed9f592f32d5e64bb81_83,PT16H25M,2,GDS,b66001f51ad00ed9f592f32d5e64bb81,KL,109,PT8H15M,83,0,AMS,2023-12-05 14:10:00,DEL,2023-12-06 01:55:00,2023-11-05 20:23:55.149435
4,759e075afd7882db4d165ca69d0bd4eb_61,PT25H15M,3,GDS,759e075afd7882db4d165ca69d0bd4eb,KL,1834,PT1H30M,61,0,BER,2023-12-05 21:10:00,AMS,2023-12-05 22:40:00,2023-11-05 20:23:55.149435


Segment key unique check

Finding out which columns in a dataframe are lists 

In [ ]:
# Assuming deduplicated_df is your DataFrame
# list_columns = []

# for column in final_df.columns:
#     if final_df[column].apply(lambda x: isinstance(x, list)).any():
#         list_columns.append(column)

# print("Columns with lists:", list_columns)

### Pricing flights

In [ ]:
offer = pd.DataFrame()

for i in range(24):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    offer = pd.concat([offer, data_to_append], ignore_index=True)

In [ ]:
pricing_df = pd.DataFrame()

for i in range(24):
    # Assuming response[i]['data'] is a dictionary
    data_to_append = pd.json_normalize(response[i]['data'], meta=['id', 'source'])
    
    # Append the data to the existing DataFrame
    pricing_df = pd.concat([pricing_df, data_to_append], ignore_index=True)

In [ ]:
pricing_df = pricing_df.drop(columns=['instantTicketingRequired','nonHomogeneous','oneWay','lastTicketingDate','lastTicketingDateTime','numberOfBookableSeats','validatingAirlineCodes','travelerPricings'],axis=1)


In [ ]:
# Extract first departure airport
pricing_df['first_dept_airport'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][0]['departure']['iataCode'])

# Extract last arrival airport
pricing_df['last_arr_airport'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][-1]['arrival']['iataCode'])

# Extract first departure time
pricing_df['first_dept_time'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][0]['departure']['at'])

# Extract last arrival time
pricing_df['last_arr_time'] = pricing_df['itineraries'].apply(lambda x: x[0]['segments'][-1]['arrival']['at'])

# Drop the 'itineraries' column if no longer needed
pricing_df.drop(columns=['itineraries'], inplace=True)

In [ ]:
def generate_unique_id(row):
    unique_string = f"{row['id']}_{row['first_dept_airport']}_{row['last_arr_airport']}_{row['first_dept_time']}_{row['last_arr_time']}"
    unique_id = hashlib.md5(unique_string.encode()).hexdigest()
    return unique_id

pricing_df['unique_id_fk'] = pricing_df.apply(generate_unique_id, axis=1)

#### Remove "price" and dots from column names


In [ ]:
pricing_df = pricing_df.rename(columns=lambda x: x.replace('price.', '').replace('price', ''))
pricing_df = pricing_df.rename(columns=lambda x: x.replace('pricingOptions.', '').replace('price', ''))

In [ ]:
# Split 'additionalServices' into two columns
pricing_df = pricing_df.drop(['additionalServices','fees','type','id','source','first_dept_time','last_arr_time','last_arr_airport','first_dept_airport'], axis=1)

In [ ]:

pricing_df['incremental_day'] =now
pricing_df['incremental_day'] = pd.to_datetime(pricing_df['incremental_day'])


In [ ]:
pricing_df.rename(columns={'grandTotal': 'grand_total','fareType': 'fare_type','includedCheckedBagsOnly': 'included_checkedbags_only'}, inplace=True)

In [ ]:
pricing_df.head()

,currency,total,base,grand_total,fare_type,included_checkedbags_only,unique_id_fk,incremental_day
0,EUR,479.48,259.00,479.48,[PUBLISHED],False,7cbbb21d4e8e94e90668e6ab656eaaf2,2023-11-05 20:23:55.149435
1,EUR,479.48,259.00,479.48,[PUBLISHED],False,b66001f51ad00ed9f592f32d5e64bb81,2023-11-05 20:23:55.149435
2,EUR,479.48,259.00,479.48,[PUBLISHED],False,759e075afd7882db4d165ca69d0bd4eb,2023-11-05 20:23:55.149435
3,EUR,479.48,259.00,479.48,[PUBLISHED],False,abd39e59d1bd6c1c39533f08076862ea,2023-11-05 20:23:55.149435
4,EUR,479.48,259.00,479.48,[PUBLISHED],False,9bad770401e165eaaef6af4522d9e5c8,2023-11-05 20:23:55.149435


In [ ]:
credentials=config.pg_credentials
db = DBManager(credentials=credentials)
db.create_table_from_df(df=offer_df,table_name='flight_offers')
db.create_table_from_df(df=itineraries_df,table_name='itineraries')
db.create_table_from_df(df=pricing_df,table_name='pricing')

INFO-db_manager:27-CONNECTED
INFO-db_manager:73-Dumped to DB
INFO-db_manager:73-Dumped to DB
INFO-db_manager:73-Dumped to DB
